<a href="https://colab.research.google.com/github/Rahullnitp/Restaurant_recommender/blob/master/recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd /content/drive/My Drive/recommender

/content/drive/My Drive/recommender


In [0]:
from google.colab import files
uploaded=files.upload()

In [0]:
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [0]:
# !kaggle datasets download -d yelp-dataset/yelp-dataset

In [0]:
import numpy as np 
import pandas as pd
# import seaborn as sns 
import re
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
# import matplotlib.pyplot as plt
# %matplotlib inline
import gc
import json

In [0]:
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer

###Data_Preprocessing

restaurant_data

In [0]:
# data_restaurant=pd.read_json('/content/drive/My Drive/recommender/yelp_academic_dataset_business.json',lines=True)

In [0]:
# data_restaurant.columns

In [0]:
# data_restaurant.shape

In [0]:
# data_restaurant=data_restaurant.rename(columns = {'review_count': 'RatingCount'})

In [0]:
#  data_restaurant=data_restaurant.dropna(subset=['categories'])

In [0]:
# data_restaurant=data_restaurant[['business_id', 'name', 'city', 'stars', 'RatingCount', 'categories']]

In [0]:
# data_restaurant.to_csv('df.csv')

In [0]:
# data_review=pd.read_json('/content/drive/My Drive/recommender/yelp_academic_dataset_review.json',lines=True,chunksize=100000)

In [0]:
# %%time
# chunk_list=[]
# for chunk in data_review:
#   chunk_list.append(chunk)
# df_concat1=pd.concat(chunk_list)

In [0]:
# df_concat1.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [0]:
# yelp_data1 = df_concat1[['business_id', 'user_id', 'stars', 'text']]

In [0]:
#  yelp_data2 = pd.merge(df[['categories','business_id','city']], yelp_data1, on='business_id')

In [0]:
# yelp_data2.shape

(115013, 6)

In [0]:
# yelp_data2['user_id'].value_counts()

In [0]:
# counts = yelp_data2['user_id'].value_counts()

# data = yelp_data2[~yelp_data2['user_id'].isin(counts[counts < 2].index)]

In [0]:
# data.shape

(43160, 6)

In [0]:
# data['business_id'].value_counts()

In [0]:
# data.to_csv('yelpData.csv')

In [0]:
databusiness=pd.read_csv('/content/drive/My Drive/recommender/df.csv')

In [0]:
databusiness['city']=databusiness['city'].str.lower()

In [0]:
databusiness.head()

,Unnamed: 0,business_id,name,city,stars,RatingCount,categories,Airport
0,1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,ranchi,2.5,128,"Specialty Food, Restaurants, Dim Sum, Imported...",Birsa Munda International Airport
1,2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,guwahati,4.0,170,"Sushi Bars, Restaurants, Japanese",Borjhar Airport
2,11,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,car nicobar,3.0,18,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Car Nicobar Airport
3,13,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,chandigarh,4.0,16,"Italian, Restaurants, Pizza, Chicken Wings",Chandigarh Airport
4,14,-K4gAv8_vjx8-2BxkVeRkA,Baby Cakes,chennai,3.0,7,"Bakeries, Food",Chennai International Airport


In [0]:
yelp_data=pd.read_csv('/content/drive/My Drive/recommender/yelpData.csv')

In [0]:
yelp_data=yelp_data.dropna()

In [0]:
yelp_data['city']=yelp_data['city'].str.lower()

In [0]:
yelp_data.head(2)

,Unnamed: 0,categories,business_id,city,user_id,stars,text
0,0,"Specialty Food, Restaurants, Dim Sum, Imported...",QXAEGFB4oINsVuTFxEYKFQ,ranchi,2K62MJ4CJ19L8Tp5pRfjfQ,3,My girlfriend and I went for dinner at Emerald...
1,1,"Specialty Food, Restaurants, Dim Sum, Imported...",QXAEGFB4oINsVuTFxEYKFQ,ranchi,A0kENtCCoVT3m7T35zb2Vg,3,We've always been there on a Sunday so we were...


In [0]:

# yelp_data['city']=yelp_data['city'].astype(object)

In [0]:
yelp_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43160 entries, 0 to 43159
Data columns (total 7 columns):
Unnamed: 0     43160 non-null int64
categories     43160 non-null object
business_id    43160 non-null object
city           43160 non-null object
user_id        43160 non-null object
stars          43160 non-null int64
text           43160 non-null object
dtypes: int64(2), object(5)
memory usage: 2.6+ MB


In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import string
from nltk.corpus import stopwords
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [0]:
def text_process(mess):                                                                                                                                                                                                                                                                              
  
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]                #  remove  (!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~) if any

    nopunc = ''.join(nopunc)                                                         # Join the characters again to form the string.
    
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])          # Now just remove any stopwords in text and return the processed text.

In [0]:
yelp_data['text'] = yelp_data['text'].apply(text_process)

In [0]:
yelp_data =yelp_data.sample(frac=1).reset_index(drop=True)  # for sampling the dataset

In [0]:
yelp_data['city'].unique()

array(['Mumbai', 'Chennai', 'Cuddapah', 'Car Nicobar', 'Kochi',
       'Cooch Behar', 'Guwahati', 'Aurangabad', 'Ranchi', 'Chandigarh'],
      dtype=object)

In [0]:
yelp_data.head()

,Unnamed: 0,categories,business_id,city,user_id,stars,text
0,108905,"Restaurants, Asian Fusion, Sushi Bars, Japanese",Yiw2jUJzUdFmglHZgj4RaQ,Mumbai,zyMB50UtmEuQf-1VUKxZxQ,4,Good food reasonable price actually enjoyed ch...
1,11784,"Vietnamese, Restaurants",AMN3Sgns9tB2x-b3eWk0qQ,Chennai,xovKY9oB8s-NjKz_I1RpHQ,3,looking authentic Vietnamese Scottsdale go cra...
2,61122,"Sandwiches, Fast Food, Salad, Restaurants, Chi...",C2T7ecxqsVu99PhSqT3c3g,Cuddapah,lX_j1NZA0xpJ9E1tu7SsIQ,4,tried salads first time pretty good get fried ...
3,50464,"Asian Fusion, Vegetarian, Vegan, Mexican, Rest...",a9aW5e731lplWGHUZ02-zQ,Car Nicobar,fCy0RLrVg6xyhaNRpZtBIA,5,liked original location location even better L...
4,52228,"Middle Eastern, Restaurants, Mediterranean, Pe...",Er2XAYN8l1BadlYFwiPb9g,Mumbai,kNzzteRJy5jBOX8rM_uJOA,5,mood try something different thought wed stop ...


In [0]:
yelp_data_test=yelp_data[:1000]

In [0]:
yelp_data=yelp_data[1000:]

In [0]:
yelp_data.shape

(42160, 7)

In [0]:
# yelp_data.to_csv('final_yelp_data.csv')

In [0]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [0]:
userid_df

,user_id,text
1000,Wxd-EKF2Z4RHZUmwQ6Eeew,frozen hot chocolate double chocolate flavor s...
1001,KE3fPgfxEmfsutKxMlssvQ,Mrs love place Owner super friendly service al...
1002,1V5tTGQbhaCBbmH-EVT7KQ,definitely fan food good service quick friendl...
1003,-50XWnmQGqBgEI-9ANvLlg,review month overdue Experience Came group fol...
1004,QuZbJquRtbY9O9JrB9NpxQ,Tandoori Times Volume 1 Issue 1 Number 1 Limit...
...,...,...
43155,C5HjsWQz-Algu8ej8DnjSA,Ive come many time following first reviewevery...
43156,ZQ3fwa4MIzJtkzQ1Kmc-sA,ordered Blackberry Sage Tea Latte Almond Milk ...
43157,zcmCg2nsT0pBinXiVEA5Wg,pretty decent meal prices reasonable strip Cap...
43158,KcjLcdus-XWt6E9uJeHONA,place awesome actually looking place served bo...


In [0]:
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
business_df = business_df.groupby('business_id').agg({'text': ' '.join})

In [0]:
# import pickle
# output = open('user_business_df.pkl', 'wb')
# pickle.dump(userid_df,output,protocol=2)
# pickle.dump(business_df,output,protocol=2)
# output.close()

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer=TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize)

In [0]:
X=vectorizer.fit_transform(userid_df['text'])

In [0]:
print(len(vectorizer.get_feature_names()))

72294


In [0]:
sentence=[' '.join(userid_df['text'])]

In [0]:
len(sentence)

1

In [0]:
wordlist=sentence[0].split()

In [0]:
wordfreq=[]
for w in wordlist:
  wordfreq.append(wordlist.count(w))

In [0]:
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=3200)

In [0]:
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])
userid_vectors.shape

(14798, 3200)

In [0]:
#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=3200)

In [0]:
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])
businessid_vectors.shape

(1866, 3200)

In [0]:
yelp_data.columns

Index(['Unnamed: 0', 'categories', 'business_id', 'city', 'user_id', 'stars',
       'text'],
      dtype='object')

In [0]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])
userid_rating_matrix 

business_id,-1xuC540Nycht_iWFeJ-dw,-K4gAv8_vjx8-2BxkVeRkA,-R2z2womJA-NCA2Pbz181g,0-jqS5y4D-6Zohb3Q375ZQ,02eESJZNF3qigrJ3GNRgIQ,036SeqOKaKx89Liw5RDKqA,06Agn-DDxhc1A1u935l93Q,096iGHoQ-UImxUExuyqlZA,09pf_DnQV5Y5y2reoIPXlw,09pjh8dL9AavSLHkdKT4VQ,0DapwcHS0YWNeHKptURSCA,0FVOh4aVtEBR_N5XwBNLRw,0G83H_zoum-Q4bWU2olTqQ,0IfrcGbFzQvn9w8bl9GKDg,0LPVkxZ-Ab9Jx6_ytA-Yew,0NRW_aZUWvDsjh58fBq8OA,0P9PCKJBC3TY9Ifcw4Y7Mw,0TRFt-MVF43hGP8YD7O2sw,0VDcJitRHh5uJnkjymu_6A,0WAL6O1YvcJbeDRYeRm34w,0W_6DPifCQwGU5N_LaiE9g,0ZN2MfHyjNIkCx7qJvVhDg,0cDKiKwsE6NAlfhx-cAWkA,0gTTGuA6uEmMhLhfJi0hqA,0i95sgY7pzYW9k88SOVJ8A,0jRcmkgwrjsPtDOeqLJweg,0kKtYDKvvcQckR4vRmfh5Q,0lFAD4CuWcQVl9u3-lRaCQ,0lUhcajLmtq3H4h9xiiabQ,0lm2kI7hkRKOYQKeYr3Pxg,0n2us6pzfJ4UK9qy1NWYOg,0nquGuiW0SONs3R89gSi1Q,0pm-hpu8Rkfa16tLjWb-GQ,0rN_o7_IfshAHQwzMUK8nA,0sE5o_dunOdUMvXVsaJSsA,0tMhUM4FJy-onBd8ppUDng,0txNpCq2YunZSL7jHIiGlA,0u4Lmqi2hPCwwxJMbA7-ng,0yHY4FHCTOIshPiPc9EYBQ,108H_8aEgOH5uz_C9hBgUQ,...,yPIHOfXNz4LNTWFjCTFWgA,yPzcnXlZ4VfCqPrXgwbS9A,yW9IIc1cBIlFlbOG_1pp_Q,yWwIUmeenyAO7nwc7U0U5A,yYkvonvBGZB6UMnpzr6Ryw,yae3VHS39BBaLGBUn94sHQ,yb7ZtgMWSZboG_sSUx9g2A,yc7ukSQi61NMXYIyHyobgQ,ydoXmW_5p_MgyoWs-dEK8Q,yex6vCCeLKsnWiNW5aGrLA,ygLcYFoKRP3PcUXcDKnz0w,yhT2iTPtvQttZxYS9cNF0A,ylC66QZbG0lQbggJFQYaqw,ynbbMfhYHJ3a1Oa70Gibig,ypMf1Ma5b-WoiVmT22deoA,ysfeDcLAYk_MVzfBmyiSTg,yt1C94WXeUokPsjyqWWaiQ,yuhnMOLGmXHhQxyp1V_rBw,yvqMh9kTv3Kx-tocc0DFBQ,yxAw_NtCqtvFPXGq4C1f5A,z0DImJOdVrSF8BmgmeMy5g,z1RYEzRQzhBKw0Pez3xSHA,z2LzzuzC8BjQcojQjd4tDQ,z3csSfd-UBMP1Gk6zHY_AA,z4CYPDxL_L74GnaD35a9Yw,z8H4Mdzl4jS9pYswj6Jf9w,zElnVlq5To9UoNw4KmoYWQ,zKSKmkBxfrltJau_4s1ZHA,zLHOG9ty0OcXSQ6iy2BwwA,zPAMOTc_VEfgPxseUtWw-w,zUHIDqm_UKdnSygmWKtyRg,zVvekQAUjhZpys3xzcCGaA,zYzYmPLViy5Q6u7rmL8AYA,z_BS6qapXMX1fA4ZLZ3jCA,zfNKdsuT7bWWN-f3v5Rj_g,zfOum9_rEhaLAqoAV3nYaQ,zgAh21YWYaZUSEGi_E5SVg,zgtgMlvFk5Ilrs1yj78tdA,zkk1ieQQnWtu9TLJukXC0g,zzwhN7x37nyjP0ZM8oiHmw
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--2HUmLkcNHZp0xw6AMBPg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--BumyUHiO_7YsHurb9Hkw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--D8T8lqv2YnX8BebVQBdg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Qh8yKWAvIP4V4K8ZPfHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-05XqtNjcBq19vh2CVJN8g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [0]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())

In [0]:
# pickle_in = open("/content/drive/My Drive/recommender/yelp_recommendation.pkl","rb")
# userid_vectors= pickle.load(pickle_in)
# businessid_vectors=pickle.load(pickle_in)
# pickle_in.close()

In [0]:
P

,1,10,100,1015,11,12,13,14,15,150,16,17,18,19,1st,2,20,200,23,24,25,250,2nd,3,30,300,32,34,35,350,3rd,4,40,45,4th,5,50,55,6,60,...,worried,worry,worse,worst,worth,worthy,would,wouldve,wow,wrap,wrapped,wraps,write,writing,written,wrong,wrote,wtf,wynn,ya,yay,yeah,year,years,yellow,yelp,yelpers,yes,yesterday,yet,yogurt,yonge,york,young,yum,yummy,zero,zucchini,à,était
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--2HUmLkcNHZp0xw6AMBPg,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.153202,0.0,0.103130,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
--BumyUHiO_7YsHurb9Hkw,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.038635,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
--D8T8lqv2YnX8BebVQBdg,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.053572,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.074914,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.081660,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.046757,0.0,0.000000,0.076031,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
--Qh8yKWAvIP4V4K8ZPfHA,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.094871,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.043559,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
-05XqtNjcBq19vh2CVJN8g,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.090589,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.119181,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyjyUDOW9IhMwNQ-PubOqw,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.164492,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
zzPvEodjvLPe-5NvV0U_cg,0.092133,0.029552,0.042705,0.0,0.0,0.0,0.0,0.0,0.034470,0.05367,0.0,0.0,0.0,0.0,0.0,0.025100,0.0,0.0,0.0,0.0,0.040543,0.0,0.000000,0.025636,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.056332,0.0,0.04034,0.0,0.026272,0.0,0.0,0.0,0.054992,...,0.0,0.0,0.0,0.0,0.077594,0.0,0.104467,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.034031,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0

In [0]:
def matrix_factorization(R, P, Q, steps=40, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        err=0
        N=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    err= err + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)
                    N=N+1
        e=err/N
        if e<0.001:
            print('error reduced')
            break
        if(step%1==0):
          output = open('P_Q.pkl', 'wb')
          pickle.dump(P,output)
          pickle.dump(Q,output)
          output.close()
          print(step)
          print(e)
    return P,Q

In [0]:
import pickle

In [0]:
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=50, gamma=0.01,lamda=0.01)

In [0]:

%%time
# P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.01,lamda=0.02)

In [0]:
Q.iloc[0].sort_values(ascending=False).head(10)

aloha    0.775763
hash     0.684916
like     0.667090
kalua    0.619905
chow     0.615568
pork     0.551984
food     0.526274
spam     0.513472
back     0.481753
good     0.451520
Name: 096iGHoQ-UImxUExuyqlZA, dtype: float64

In [0]:
import pickle
pickle_in = open("/content/drive/My Drive/recommender/P_Q_new.pkl","rb")
P= pickle.load(pickle_in)
Q=pickle.load(pickle_in)

pickle_in.close()

In [0]:
# # Store P, Q and vectorizer in pickle file
# import pickle
# output = open('yelp_recommendation.pkl', 'wb')
# pickle.dump(P,output,protocol=2)
# pickle.dump(Q,output,protocol=2)
# pickle.dump(userid_vectorizer,output,protocol=2)
# output.close()

In [0]:
def weighted_rating(x,C,m):
  v=x['RatingCount']
  R=x['stars']
  # C=data_business['stars'].mean()
  # m=data_business['RatingCount'].quantile(0.90)
  w_r=(v/(v+m))*R + (m/(v+m))*C
  # w_r.sort_values(by=['score'], inplace=True)
  return w_r

In [0]:
def User_Rating(user_id,CITY):
  flag=0
  # words = userid_df.loc[user_id]['text']
  # test_df= pd.DataFrame([words], columns=['text'])
  # test_df['text'] = test_df['text'].apply(text_process)
  # test_vectors = userid_vectorizer.transform(test_df['text'])  
  # test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())
  # print(test_v_df)
  if user_id in P.index:
    test_v_df1=pd.DataFrame(P.loc[P.index==user_id])
    test_v_df1=test_v_df1.reset_index(drop=True)

    predictItemRating=pd.DataFrame(np.dot(test_v_df1.loc[0],Q.T),index=Q.index,columns=['Rating'])
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])
  # print(topRecommendations)
    to_return=[]
    for i in topRecommendations.index:
      if flag==5:
        break
      if databusiness[databusiness['business_id']==i]['city'].any()==CITY.lower():
      # print(data_business[data_business['business_id']==i]['name'].iloc[0])
      # print('Categories : '+data_business[data_business['business_id']==i]['categories'].iloc[0])
      # print(str(data_business[data_business['business_id']==i]['stars'].iloc[0])+ ' '+str(data_business[data_business['business_id']==i]['review_count'].iloc[0]))
      # print('')
        flag=flag+1
        to_return.append({
          'Restaurant':databusiness[databusiness['business_id']==i]['name'].iloc[0],
          'Categories':databusiness[databusiness['business_id']==i]['categories'].iloc[0],
          'rating':databusiness[databusiness['business_id']==i]['stars'].iloc[0],
          'RatingCount':databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]
                 })      
    
    if flag==0:
      print('no restaurant in the city')
    return to_return
  else:
    weighted_data=databusiness.loc[databusiness['city']==CITY.lower()]
    C=weighted_data['stars'].mean()
    m=weighted_data['RatingCount'].quantile(0.90)
    qualified_rest=weighted_data.copy().loc[weighted_data['RatingCount'] >= m]  

    qualified_rest['score'] =weighted_rating(qualified_rest,C,m)
    # qualified_rest['score'] = qualified_rest.apply(weighted_rating, axis=1)
    qualified_rest.sort_values(by=['score'], inplace=True,ascending=False)
    qualified_rest=pd.DataFrame(qualified_rest).drop(['business_id'],axis=1).reset_index(drop=True)
    top_5=qualified_rest[:5]
    return top_5

In [0]:
t=User_Rating('xd0ZjCAAOoBnjfZnDKGlEA','kochi')

In [0]:
t

[{'Categories': 'Caterers, Hot Dogs, Restaurants, Vegetarian, American (Traditional), Event Planning & Services',
  'RatingCount': 380,
  'Restaurant': "JJ's Red Hots - Dilworth",
  'rating': 4.0},
 {'Categories': 'Indian, Restaurants',
  'RatingCount': 497,
  'Restaurant': 'Marigold Maison- Paradise Valley',
  'rating': 4.0},
 {'Categories': 'Burgers, Restaurants, American (New), Bars, American (Traditional), Nightlife',
  'RatingCount': 102,
  'Restaurant': 'The Turnhouse Grille',
  'rating': 4.5},
 {'Categories': 'American (Traditional), Burgers, Restaurants',
  'RatingCount': 114,
  'Restaurant': 'Fuddruckers',
  'rating': 3.5},
 {'Categories': 'Herbs & Spices, Food, Specialty Food',
  'RatingCount': 90,
  'Restaurant': 'Savory Spice Shop',
  'rating': 4.5}]

In [0]:
def new_user(CITY,words):
  flag=0
  # words = input("Enter your value: ") 
  test_df= pd.DataFrame([words], columns=['text'])
  test_df['text'] = test_df['text'].apply(text_process)
  test_vectors = userid_vectorizer.transform(test_df['text'])  
  test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())
  to_return=[]
  predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
  topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])
  # print(topRecommendations)

  for i in topRecommendations.index:
    if flag==5:
        break
    if databusiness[databusiness['business_id']==i]['city'].any()==CITY.lower():
      # print(databusiness[databusiness['business_id']==i]['name'].iloc[0])
      # print(databusiness[databusiness['business_id']==i]['categories'].iloc[0])
      # print(str(databusiness[databusiness['business_id']==i]['stars'].iloc[0])+ ' '+str(databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]))
      # print('')
      flag=flag+1
      to_return.append({
          'Restaurant':databusiness[databusiness['business_id']==i]['name'].iloc[0],
          'Categories':databusiness[databusiness['business_id']==i]['categories'].iloc[0],
          'rating':databusiness[databusiness['business_id']==i]['stars'].iloc[0],
          'RatingCount':databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]
                 })  
  return to_return


In [0]:
t=new_user('Kochi','food')
t

[{'Categories': 'Fast Food, Burgers, Restaurants',
  'RatingCount': 6,
  'Restaurant': "Wendy's",
  'rating': 2.0},
 {'Categories': 'Restaurants, Food, Cafes',
  'RatingCount': 14,
  'Restaurant': 'Vals Getaway DES Cafeteria',
  'rating': 3.0},
 {'Categories': 'Food, Nightlife, Bars, Sports Bars, Beer, Wine & Spirits, Restaurants',
  'RatingCount': 82,
  'Restaurant': 'Tailgaters Sports Bar & Grill',
  'rating': 3.0},
 {'Categories': 'Burgers, Restaurants, American (Traditional)',
  'RatingCount': 115,
  'Restaurant': 'Fuddruckers',
  'rating': 3.0},
 {'Categories': 'Hotels & Travel, American (Traditional), Bed & Breakfast, Restaurants, Breakfast & Brunch, Salad, Comfort Food, American (New)',
  'RatingCount': 19,
  'Restaurant': 'Bob Evans',
  'rating': 3.5}]

In [0]:
err=0
N=0
for i in range(yelp_data_test.shape[0]):

  user=yelp_data_test.iloc[i]['user_id']
  business=yelp_data_test.iloc[i]['business_id']
  if (user in P.index) & (business in Q.index):
    N=N+1
    err=err+abs((yelp_data_test.iloc[i]['stars'])-np.dot(P.loc[user],Q.loc[business]))
    print(err)
    # print(yelp_data_test.iloc[i]['stars'])
    # print(P.loc[user])
    # print(Q.loc[business])
    print(N)
    # print('\n')
e=err/N

In [0]:
err=0
N=0
for i in range(yelp_data_test.shape[0]):

  user=yelp_data_test.iloc[i]['user_id']
  business=yelp_data_test.iloc[i]['business_id']
  if (user in P.index) & (business in Q.index):
    N=N+1
    err=err+pow((yelp_data_test.iloc[i]['stars'])-np.dot(P.loc[user],Q.loc[business]),2)
    print(err)
    # print(yelp_data_test.iloc[i]['stars'])
    # print(P.loc[user])
    # print(Q.loc[business])
    print(N)
    # print('\n')
e=err/N

In [0]:
e

0.13119333999257354

In [0]:
def new_user(CITY):
  words = input("Enter your value: ") 
  test_df= pd.DataFrame([words], columns=['text'])
  test_df['text'] = test_df['text'].apply(text_process)
  test_vectors = userid_vectorizer.transform(test_df['text'])  
  test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

  predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
  topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])
  # print(topRecommendations)

  for i in topRecommendations.index:
    if databusiness[databusiness['business_id']==i]['city'].any()==CITY:
      print(databusiness[databusiness['business_id']==i]['name'].iloc[0])
      print(databusiness[databusiness['business_id']==i]['categories'].iloc[0])
      print(str(databusiness[databusiness['business_id']==i]['stars'].iloc[0])+ ' '+str(databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]))
      print('')


In [0]:
new_user('Kochi')

In [0]:
def User_Rating(user_id,CITY):
  flag=0
  # words = userid_df.loc[user_id]['text']
  # test_df= pd.DataFrame([words], columns=['text'])
  # test_df['text'] = test_df['text'].apply(text_process)
  # test_vectors = userid_vectorizer.transform(test_df['text'])  
  # test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())
  # print(test_v_df)
  if user_id in P.index:
    test_v_df1=pd.DataFrame(P.loc[P.index==user_id])
    test_v_df1=test_v_df1.reset_index(drop=True)

    predictItemRating=pd.DataFrame(np.dot(test_v_df1.loc[0],Q.T),index=Q.index,columns=['Rating'])
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])
  # print(topRecommendations)
    to_return=[]
    for i in topRecommendations.index:
      if flag==5:
        break
      if databusiness[databusiness['business_id']==i]['city'].any()==CITY:
      # print(data_business[data_business['business_id']==i]['name'].iloc[0])
      # print('Categories : '+data_business[data_business['business_id']==i]['categories'].iloc[0])
      # print(str(data_business[data_business['business_id']==i]['stars'].iloc[0])+ ' '+str(data_business[data_business['business_id']==i]['review_count'].iloc[0]))
      # print('')
        flag=flag+1
        to_return.append({
          'Restaurant':databusiness[databusiness['business_id']==i]['name'].iloc[0],
          'Categories':databusiness[databusiness['business_id']==i]['categories'].iloc[0],
          'rating':databusiness[databusiness['business_id']==i]['stars'].iloc[0],
          'rating_count':databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]
                 })      
    
    if flag==0:
      print('no restaurant in the city')
    return to_return
  else:
    print('user not in the list')

In [0]:
t

[{'Categories': 'Caterers, Hot Dogs, Restaurants, Vegetarian, American (Traditional), Event Planning & Services',
  'Restaurant': "JJ's Red Hots - Dilworth",
  'rating': 4.0,
  'rating_count': 380},
 {'Categories': 'Indian, Restaurants',
  'Restaurant': 'Marigold Maison- Paradise Valley',
  'rating': 4.0,
  'rating_count': 497},
 {'Categories': 'Burgers, Restaurants, American (New), Bars, American (Traditional), Nightlife',
  'Restaurant': 'The Turnhouse Grille',
  'rating': 4.5,
  'rating_count': 102},
 {'Categories': 'American (Traditional), Burgers, Restaurants',
  'Restaurant': 'Fuddruckers',
  'rating': 3.5,
  'rating_count': 114},
 {'Categories': 'Herbs & Spices, Food, Specialty Food',
  'Restaurant': 'Savory Spice Shop',
  'rating': 4.5,
  'rating_count': 90}]

In [0]:
t=User_Rating('xd0ZjCAAOoBnjfZnDKA','Kochi')

In [0]:
t

,Unnamed: 0,name,city,stars,RatingCount,categories,Airport,score
0,25,Maria's Mexican Restaurant & Bakery,Kochi,4.5,184,"Mexican, Restaurants, Patisserie/Cake Shop, Fo...",Cochin International Airport,4.128621
1,1534,Fuego Taco Shop,Kochi,4.5,143,"Food, Mexican, Restaurants",Cochin International Airport,4.069741
2,2088,Songbird Coffee & Tea House,Kochi,4.5,136,"Coffee & Tea, Cafes, Food, Tea Rooms, Restaura...",Cochin International Airport,4.057770
3,1664,Marigold Maison- Paradise Valley,Kochi,4.0,497,"Indian, Restaurants",Cochin International Airport,3.912724
4,157,JJ's Red Hots - Dilworth,Kochi,4.0,380,"Caterers, Hot Dogs, Restaurants, Vegetarian, A...",Cochin International Airport,3.892121


In [0]:
t=User_Rating('-2UOdDlzg7XLuUuyOrldhA','Bhopal')

In [0]:
t

In [0]:
y=databusiness.loc[databusiness['city']=='Tezu']

In [0]:
y

In [0]:
def User_Rating(user_id,CITY):
  if user_id in P.index:
    test_v_df1=pd.DataFrame(P.loc[P.index==user_id])
    test_v_df1=test_v_df1.reset_index(drop=True)

    predictItemRating=pd.DataFrame(np.dot(test_v_df1.loc[0],Q.T),index=Q.index,columns=['Rating'])
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])
  # print(topRecommendations)
    to_return=[]
    for i in topRecommendations.index:
      if databusiness[databusiness['business_id']==i]['city'].any()==CITY:

        flag=flag+1
        to_return.append({
          'Restaurant':databusiness[databusiness['business_id']==i]['name'].iloc[0],
          'Categories':databusiness[databusiness['business_id']==i]['categories'].iloc[0],
          'rating':databusiness[databusiness['business_id']==i]['stars'].iloc[0],
          'rating_count':databusiness[databusiness['business_id']==i]['RatingCount'].iloc[0]
                 })      
    
    if flag==0:
      print('no restaurant in the city')
    return to_return
  else:
    print('user not in the list')

In [0]:
e

0.04985620191052029

In [0]:
# data_business1=pd.read_json('/content/drive/My Drive/recommender/yelp_academic_dataset_business.json',lines=True)

In [0]:
# data_business1=data_business1.rename(columns = {'review_count': 'RatingCount'})

In [0]:
#  data_business1=data_business1.dropna(subset=['categories'])

In [0]:
# data_business1 = data_business1[data_business1['categories'].str.contains("Food|Coffee|Tea|Restaurants|Bakeries|cake|Donuts|Bakeries|Food|Sandwiches|Ice Cream|Yogurt|Breakfast|Brunch|Burgers|Dinners|Salad|Pizza")]

In [0]:
# data_business1.shape

(75460, 14)

In [0]:
# data_business1

In [0]:
# df=data_business1[:2000]

In [0]:
# df=df[['business_id', 'name', 'city', 'stars', 'RatingCount', 'categories']]

In [0]:
# df=pd.read_csv('/content/drive/My Drive/recommender/df.csv')

In [0]:
# df.head()

In [0]:
df_concat1.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [0]:
databusi=databusiness

In [0]:
databusi['city'].value_counts()

Ranchi         200
Chennai        200
Guwahati       200
Cuddapah       200
Cooch Behar    200
Kochi          200
Chandigarh     200
Car Nicobar    200
Mumbai         200
Aurangabad     200
Name: city, dtype: int64